In [51]:
import numpy as np
import pandas as pd
import requests
import json
import time

In [3]:
df = pd.read_csv('Data/ultrasignup/us_postal_codes.csv')
df.head()

,Zip Code,Place Name,State,State Abbreviation,County,Latitude,Longitude
0,501,Holtsville,New York,NY,Suffolk,40.8154,-73.0451
1,544,Holtsville,New York,NY,Suffolk,40.8154,-73.0451
2,1001,Agawam,Massachusetts,MA,Hampden,42.0702,-72.6227
3,1002,Amherst,Massachusetts,MA,Hampshire,42.3671,-72.4646
4,1003,Amherst,Massachusetts,MA,Hampshire,42.3919,-72.5248


In [12]:
# Create list of first 3 digits for each zip code

for i in range(df.shape[0]):
    df.loc[i,'first3'] = str(df.loc[i,'Zip Code'])[0:3]

In [18]:
df.drop_duplicates(subset='first3', inplace=True)
df.index = range(df.shape[0])

In [20]:
URLS = []
for i in range(df.shape[0]):
    lat = df.loc[i,'Latitude']
    lng = df.loc[i,'Longitude']
    URLS.append(f'http://ultrasignup.com/service/events.svc/closestevents?past=1&lat={lat}&lng={lng}&mi=200&mo=12&on')

In [72]:
responses = []

for i,url in enumerate(URLS):
    responses.append(requests.get(url).json())
    time.sleep(1)

In [89]:
ultra_df = pd.DataFrame()
idx = 0

for i,r in enumerate(responses):
    for j, event in enumerate(r):
        ultra_df.loc[idx, 'State'] = event['State']
        ultra_df.loc[idx, 'City'] = event['City']
        ultra_df.loc[idx, 'Event'] = event['EventName']
        ultra_df.loc[idx, 'Latitude'] = event['Latitude']
        ultra_df.loc[idx, 'Longitude'] = event['Longitude']
        ultra_df.loc[idx, 'Date'] = event['EventDate']
        idx += 1

In [91]:
ultra_df.drop_duplicates(subset='Event', inplace=True)

In [93]:
ultra_df.to_csv('Data/ultrasignup/ultrasignup.csv')